# Preliminary analysis

## Useful libraries

In [1]:
import matplotlib.font_manager as font_manager
from matplotlib.lines import Line2D
import matplotlib.ticker as mtick
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import pandas as pd
import numpy as np
import os
import sys
import re

## Data importation

In [4]:
datapath = 'C:\Users\Kelyan\OneDrive\Documents\GitHub\ada-2022-project-geksada'
yt_meta_path = os.path.join(datapath, "yt_metadata_en.jsonl.gz")
yt_meta_16_feater = os.path.join(datapath, "video_metadata2016.feather")

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (2624398762.py, line 1)

In [7]:
df_meta_16 = pd.read_feather("video_metadata2016.feather")

In [8]:
print(len(df_meta_16))
df_meta_16.head()

4130410


,index,categories,channel_id,crawl_date,dislike_count,display_id,duration,like_count,tags,upload_date,view_count
0,185,Education,UCzWrhkg9eK5I8Bm3HfV-unA,2019-10-31 20:19:36.933325,3.0,jCaqrZ8gSkY,116,11.0,"learn colors,kids colors,colors kids,count to ...",2016-05-03 00:00:00,2720.0
1,2658,Music,UCzWfqsjlvzAOUozsGFR3kbg,2019-10-31 15:42:09.015401,28.0,WVapFh7cx3M,142,184.0,"En Av oss,one of us,project,Gutterommet,music,...",2016-06-06 00:00:00,4360.0
2,3862,Music,UCzWdpFOflXTOk5Gsi2aJ67g,2019-11-13 21:46:41.215349,0.0,Qr8sptZ2vyA,266,25.0,"Atis Freivalds,Paradox,scary music,dark ambien...",2016-12-27 00:00:00,697.0
3,3863,Music,UCzWdpFOflXTOk5Gsi2aJ67g,2019-11-13 21:46:41.849022,8.0,ZA5EW-DKS6M,473,188.0,"Psionic Asylum,Mental Jail,scary music,horror ...",2016-12-26 00:00:00,10387.0
4,3864,Music,UCzWdpFOflXTOk5Gsi2aJ67g,2019-11-13 21:46:42.494555,2.0,bXlmyO-iW8g,192,35.0,"Clau||M,Dioxin,creepy msuic,epic horror music,...",2016-11-06 00:00:00,1702.0


## Selecting artistic videos

In [11]:
# Formatting the tags column in order to do exact word matching
df_meta_16['tags'] = df_meta_16['tags'].apply(str.lower)
df_meta_16['tags'] = df_meta_16['tags'].apply(str.replace, args=(';',' ',))
df_meta_16['tags'] = df_meta_16['tags'].apply(str.replace, args=(',',' ',))
df_meta_16['tags'] = df_meta_16['tags'].apply(str.replace, args=('(',' ',))
df_meta_16['tags'] = df_meta_16['tags'].apply(str.replace, args=(')',' ',))
df_meta_16['tags'] = df_meta_16['tags'].apply(str.replace, args=('&',' ',))
df_meta_16['tags'] = df_meta_16['tags'].apply(str.replace, args=('|',' ',))
df_meta_16['tags'] = df_meta_16['tags'].apply(str.split)

In [12]:
df_meta_16.head()

,index,categories,channel_id,crawl_date,dislike_count,display_id,duration,like_count,tags,upload_date,view_count
0,185,Education,UCzWrhkg9eK5I8Bm3HfV-unA,2019-10-31 20:19:36.933325,3.0,jCaqrZ8gSkY,116,11.0,"[learn, colors, kids, colors, colors, kids, co...",2016-05-03 00:00:00,2720.0
1,2658,Music,UCzWfqsjlvzAOUozsGFR3kbg,2019-10-31 15:42:09.015401,28.0,WVapFh7cx3M,142,184.0,"[en, av, oss, one, of, us, project, gutteromme...",2016-06-06 00:00:00,4360.0
2,3862,Music,UCzWdpFOflXTOk5Gsi2aJ67g,2019-11-13 21:46:41.215349,0.0,Qr8sptZ2vyA,266,25.0,"[atis, freivalds, paradox, scary, music, dark,...",2016-12-27 00:00:00,697.0
3,3863,Music,UCzWdpFOflXTOk5Gsi2aJ67g,2019-11-13 21:46:41.849022,8.0,ZA5EW-DKS6M,473,188.0,"[psionic, asylum, mental, jail, scary, music, ...",2016-12-26 00:00:00,10387.0
4,3864,Music,UCzWdpFOflXTOk5Gsi2aJ67g,2019-11-13 21:46:42.494555,2.0,bXlmyO-iW8g,192,35.0,"[clau, m, dioxin, creepy, msuic, epic, horror,...",2016-11-06 00:00:00,1702.0


In [20]:
# Words of Visual Arts
words_path_visual = "visual_arts.txt"
words_path_literary = "literary_art.txt"

visual_arts = pd.read_csv(words_path_visual)
literary_arts = pd.read_csv(words_path_literary)

visual_words = visual_arts.values
literary_words = literary_arts.values

visual_words = visual_words.flatten()
literary_words = literary_words.flatten()

print(visual_words)
print(literary_words)

['ceramics' 'draw' 'draw in' 'drawing' 'drawings' 'paint' 'painter'
 'painting' 'paintings' 'sculpt' 'sculptor' 'sculpture' 'sculptures'
 'statue' 'statues' 'statuary' 'fresco' 'printmaking' 'linoleum'
 'linocut ' 'design' 'designs' 'designer' 'designers' 'craft' 'crafts'
 'photography' 'film-making' 'architect' 'architecture' 'architectures'
 'maquette' 'maquettes' 'pottery' 'potteries' 'earthenware' 'earthenwares'
 'terracotta' 'vase' 'vases' 'clayware' 'mosaic' 'mosaics' 'carve'
 'carving' 'motifs' 'sketching' 'sketchings' 'sketched' 'doodle' 'doodles'
 'doodling' 'dooodlings' 'acrylic' 'acrylic paint' 'oil paint'
 'spray paint' 'artwork' 'artworks' 'photojournalism' 'aperture'
 'exposure']
['literatures' 'writing' 'text' 'texts' 'poetry' 'prose' 'assonance'
 'alliteration' 'rhythm' 'poem' 'poems' 'novel' 'novels' 'romance'
 'novella' 'drama' 'theatre' 'article' 'articles' 'biography'
 'biographies' 'essay' 'essays' 'pamphlet' 'abstract' 'book' 'books'
 'letter' 'letters' 'writings'

In [16]:
def check_tags_in_list(tags, word_list):
    word_present = 0
    
    # Make sets from the list of words
    x = set(tags)
    y = set(word_list)
    
    # Use intersection function of set type to quickly find which word are in both lists/sets
    z = x.intersection(y)
    
    # If the intersection is larger than 0, then there is at least a common word between both sets 
    if len(z) > 0:
        word_present = 1
    return word_present

In [21]:
df_meta_16['visual_art'] = df_meta_16['tags'].apply(lambda x : check_tags_in_list(x, visual_words))
df_meta_16['literary_art'] = df_meta_16['tags'].apply(lambda x : check_tags_in_list(x, literary_words))


In [19]:
df_meta_16.visual_art.value_counts()

0    3980941
1     149469
Name: visual_art, dtype: int64